<a href="https://colab.research.google.com/github/healerpeace/fastAI_FINALS/blob/main/finalFastAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq fastbook

In [ ]:
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.all import *
from fastai.vision.widgets import *
from pathlib import Path

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', 'c769cff323d34e6fb2364b32cd80d08c')

In [ ]:
flowers = 'rose','wildflower','gumamela', 'jasmine', 'dandelion'
path = Path('flowers')

if not path.exists():
    path.mkdir()
    for o in flowers:
        dest = (path/o)
        dest.mkdir(exist_ok = True)
        results = search_images_bing(key, f'{o} pictures')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [ ]:
fns = get_image_files(path)

In [ ]:
failed = verify_images(fns)
failed.map(Path.unlink);

In [ ]:
flowers = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=1),
    get_y=parent_label,
    item_tfms=Resize(128))
    
dls = flowers.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=6, nrows=1)

In [ ]:
flowers = flowers.new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = flowers.dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
flowers = flowers.new(item_tfms=Resize(128), batch_tfms=aug_transforms())
dls = flowers.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
flowers = flowers.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = flowers.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [1]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(10, nrows=2)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
learn.export('flowers.pkl')

In [ ]:
path = Path()
learn_inf = load_learner(path/'flowers.pkl')

In [ ]:
learn_inf.dls.vocab

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
img = PILImage.create(btn_upload.data[-1])

In [ ]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred